In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import requests
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Function to get latitude and longitude from city name
def get_coordinates(city_name):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {"q": city_name, "format": "json", "limit": 1}
    headers = {"User-Agent": "MyWeatherApp/1.0"}  # To avoid blocking
    
    response = requests.get(base_url, params=params, headers=headers)
    
    if response.status_code != 200 or not response.json():
        print("Error: Could not fetch coordinates.")
        return None
    
    data = response.json()[0]
    return float(data["lat"]), float(data["lon"])

In [3]:
# Function to get weather data
def get_weather_data(lat, lon):
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m",
        "timezone": "auto"
    }
    
    response = requests.get(weather_url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching weather data.")
        return None

In [4]:
# Ask user for city input
city = input("Enter city name: ")
coords = get_coordinates(city)

In [5]:
if coords:
    lat, lon = coords
    print(f"✔️ Found '{city}' at Latitude: {lat}, Longitude: {lon}")
    
    # Fetch and process weather data
    weather_data = get_weather_data(lat, lon)
    
    if weather_data:
        try:
            df = pd.DataFrame(weather_data["hourly"])
            df.drop(columns=["time"], inplace=True, errors="ignore")  # Drop time column
            
            print("\n📊 Processing weather data...")
            df.fillna(df.mean(), inplace=True)  # Handle missing values
            
            scaler = StandardScaler()
            df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
            
            X = df_scaled.drop(columns=["temperature_2m"])
            y = df_scaled["temperature_2m"]
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            
            model = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)
            model.fit(X_train, y_train)
            
            print("🚀 Model training complete!")
            
            # Make hourly predictions for the next 12 hours
            predictions = model.predict(X_test[:12])  # Taking first 12 samples from test set
            
            print("\n🔮 Hourly Temperature Predictions for next 12 hours:")
            for i, temp in enumerate(predictions):
                print(f"Hour {i+1}: {temp:.2f}°C")

        except Exception as e:
            print(f"⚠️ Oops! Something went wrong during processing: {e}")
else:
    print("⚠️ Could not proceed without coordinates.")

✔️ Found 'New York' at Latitude: 40.7127281, Longitude: -74.0060152

📊 Processing weather data...
🚀 Model training complete!

🔮 Hourly Temperature Predictions for next 12 hours:
Hour 1: 0.09°C
Hour 2: -0.47°C
Hour 3: -0.80°C
Hour 4: -0.47°C
Hour 5: 1.14°C
Hour 6: -0.16°C
Hour 7: -1.16°C
Hour 8: 0.68°C
Hour 9: 0.60°C
Hour 10: -0.52°C
Hour 11: 0.84°C
Hour 12: 0.68°C
